In [ ]:
!pip install tensorboard pandas
!pip install matplotlib seaborn
!pip install tensorflow

In [ ]:
import os

In [ ]:
path = '/home/asap7772/highway-env/data'
lst = []
for root, dirs, files in os.walk(path, topdown=False):
    for name in files:
      x = os.path.join(root, name)
      if 'tfevents' in x:
        lst.append(x)
        print('added: ', x)

# Plotting TensorBoard

In [ ]:
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb

In [ ]:
major_ver, minor_ver, _ = version.parse(tb.__version__).release
assert major_ver >= 2 and minor_ver >= 3, \
    "This notebook requires TensorBoard 2.3 or later."
print("TensorBoard version: ", tb.__version__)

In [ ]:
from tensorflow.python.summary.summary_iterator import summary_iterator
from collections import defaultdict
def get_data(path, prefix=''):
    data = defaultdict(list)
    for e in summary_iterator(path):
        for v in e.summary.value:
            data[v.tag].append(v.simple_value)
    return data

dcts = [get_data(x) for x in lst]
print(dcts[0].keys())

In [ ]:
import heapq

plt.figure(figsize=(20,10))


pq = []
i = 0
for dct in dcts:
    heapq.heappush(pq, (-max(dct['rollout/ep_rew_mean']), dct, i))
    i += 1

for i in range(10):
    val,dct,i = heapq.heappop(pq)
    plt.plot(dct['rollout/ep_rew_mean'], label=lst[i].split('/')[-2])
plt.legend()
plt.show()

In [ ]:
from tensorflow.python.summary.summary_iterator import summary_iterator
from collections import defaultdict

def plot_tensorboard(path, prefix=''):
    data = defaultdict(list)
    for e in summary_iterator(path):
        for v in e.summary.value:
            data[v.tag].append(v.simple_value)
    
    # Plot the dataframe
    for col in data:
        plt.figure()
        plt.plot(data[col])
        plt.title(prefix + col)
        plt.show()
    

In [ ]:
for x in lst:
    name = x.split('/')[-3] + '_' + x.split('/')[-2] + ': '
    plot_tensorboard(x, prefix=name)

In [ ]:
from tensorflow.python.summary.summary_iterator import summary_iterator
from collections import defaultdict
import wandb


def plot_wandb(path, prefix=''):
    print('loading: ', path)
    print('prefix: ', prefix)
    
    wandb.init(project="highway-env", reinit=True, settings=wandb.Settings(start_method="fork"))
    wandb.run.name = prefix
    wandb.run.save()
    
    for e in summary_iterator(path):
        to_log = {}
        for v in e.summary.value:
            to_log[v.tag] = v.simple_value
        wandb.log(to_log)

In [ ]:
for x in lst:
    name = x.split('/')[-3] + '_' + x.split('/')[-2] + ': '
    plot_wandb(x, prefix=name)